# DLO-JZ Optimisation de l'apprentissage - Jour 2

Optimisation système d'une boucle d'apprentissage *Resnet-152*.

![car](./images/optimisation.png)


## Objet du notebook

Le but de ce *notebook* est d'optimiser un code d'apprentissage d'un modèle *Resnet-50* sur *Imagenet* pour Jean Zay en implémentant :
* **TP 1** : l'optimisation du *Dataloader*
* **TP 2** : la distribution (*Data Parallelism*)


Les cellules dans ce *notebook* ne sont pas prévues pour être modifiées, sauf rares exceptions indiquées dans les commentaires. Les TP se feront en modifiant les codes `dlojz2_X.py`.

Les directives de modification seront marquées par l'étiquette **TODO** dans le *notebook* suivant.
 
Les solutions sont présentes dans le répertoire `solutions/`.

*Notebook rédigé par l'équipe assistance IA de l'IDRIS, janvier 2024*

------------------------

### Environnement de calcul

Les fonctions *python* de gestion de queue SLURM développées par l'IDRIS et les fonctions dédiées à la formation DLO-JZ sont à importer.

Le module d'environnement pour les *jobs* et la taille des images sont fixés pour ce *notebook*.

**TODO :** choisir un pseudonyme (maximum 5 caractères) pour vous différencier dans la queue SLURM pendant la formation.

In [ ]:
from idr_pytools import display_slurm_queue, gpu_jobs_submitter, search_log
from dlojz_tools import controle_technique, compare, GPU_underthehood, plot_accuracy, lrfind_plot, imagenet_starter, comm_profiler, turbo_profiler, BatchNorm_view
MODULE = 'pytorch-gpu/py3/2.3.0'
image_size = 224
account = 'for@a100'
name = 'pseudo'   ## Pseudonyme à choisir

Création d'un répertoire `checkpoints/` si cela n'a pas déjà été fait.

In [ ]:
!mkdir -p checkpoints

------------------------------------

### Gestion de la queue SLURM

Pour afficher vos jobs dans la queue SLURM :

In [ ]:
display_slurm_queue(name)

**Remarque**: cette fonction sera utilisée plusieurs fois dans ce *notebook*. Elle permet d'afficher la queue de manière dynamique, rafraichie toutes les 5 secondes. Elle ne s'arrête que lorsque la queue est vide. Si vous désirez reprendre la main sur le *notebook*, il vous suffira d'arrêter manuellement la cellule avec le bouton *stop*. Cela n'a bien sûr aucun impact sur les *jobs* soumis.

Si vous voulez retirer TOUS vos *jobs* de la queue SLURM, décommenter et exécuter la cellule suivante :

In [ ]:
#!scancel -u $USER

Si vous voulez retirer UN de vos *jobs* de la queue SLURM, décommenter, compléter et exécuter la cellule suivante :

In [ ]:
#!scancel <jobid>

--------------

### Différence entre deux scripts

Pour comparer son code avec les solutions mises à disposition, la fonction suivante permet d'afficher une page HTML contenant un différentiel de fichiers texte.

In [ ]:
s1 = "./dlojz2_2.py"
s2 = "./solutions/dlojz2_2.py"
compare(s1, s2)

Voir le résultat du différentiel de fichiers sur la page suivante (attention au spoil !) :

[compare.html](compare.html)

----------------------

## Garage - Mise à niveau

On fixe la taille d'image pour ce TP.

In [ ]:
image_size = 224

On fixe le *batch size* optimal d'après les expériences du Jour 1.

In [ ]:
bs_optim = 512

# TP2_1: Optimisation du DataLoader

Dans ce TP, on utilisera le script [dlojz2_1.py](./dlojz2_1.py) dans lequel le profiler PyTorch n'est pas implémenté. Ce script est identique à la solution du TP2_1.

Dans un premier temps, on va désactiver toutes les optimisations du DataLoader (**version sous-optimisée**). Ensuite,  nous pourrons observer l'impact de chacune des optimisations possibles en les réintégrant une par une.

### Découverte de turbo_profiler
Pour ce TP, nous avons implémenté un profiler maison léger `turbo_profiler` basé sur l'outil `Chronometer` pour visualiser le temps passé sur CPU (DataLoader) et sur GPU (le reste de l'itération). Ce profiler est moins précis mais cela nous permettra de désactiver le profiler PyTorch pour ne pas dégrader les performances et éviter de devoir ouvrir l'outil graphique TensorBoard à chaque fois pour visualiser les informations qui nous intéressent.

### Version sous-optimisée

**TODO** : lancer l'exécution sur 1 GPU et 50 itérations (`--test-nsteps 50`) sans profiling pour passer un contrôle technique qui servira de référence. Cela va prendre quelques minutes (~5min), **vous pouvez passer à la suite sans attendre la fin de l'exécution**.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['90764']

In [ ]:
display_slurm_queue(name)

### Quizz

L'éxécution étant assez longue, un quizz vous attend : [Quizz TP2_1](https://www.deepmama.com/quizz/dlojz_quizz4.html)

In [ ]:
controle_technique(jobid)

**TODO** : visualiser la sortie de `turbo_profiler`

In [ ]:
# call turbo_profiler
dataloader_trial = turbo_profiler(jobid,dataloader_info=True)

Via le turbo profiler, on va également récupérer et stocker les performances obtenues dans une DataFrame `dataloader_trials` :
* initialisation de la DataFrame :

In [ ]:
import pandas as pd
dataloader_trials = pd.DataFrame({"jobid":pd.Series([],dtype=str),
                                  "num_workers":pd.Series([],dtype=int),
                                  "pin_memory":pd.Series([],dtype=str),
                                  "non_blocking":pd.Series([],dtype=str),
                                  "prefetch_factor":pd.Series([],dtype=int),
                                  "persistent_workers":pd.Series([],dtype=str),
                                  "drop_last":pd.Series([],dtype=str),
                                  "loading_time":pd.Series([],dtype=float),
                                  "CPU_memory_usage(GB)":pd.Series([],dtype=float)})

* stockage du résultat précédent dans la *DataFrame* :

In [ ]:
# store result in "dataloader_trials" DataFrame
dataloader_trials = pd.concat([dataloader_trials,dataloader_trial], ignore_index=True)

* visualisation du contenu de la *DataFrame* :

----------------------

In [ ]:
# afficher le tableau récapitulatif, trier par ordre croissant du LOADING_TIME
dataloader_trials.sort_values("loading_time")

### Exploration des paramètres d'optimisation du DataLoader
L'objectif de ce TP est de réduire le temps passé sur CPU par le DataLoader.

Pour cette étude, on continue à lancer les exécutions sur 1 GPU et 16 itérations seulement (`--test-nsteps 16`) pour avancer plus rapidement. 

Les différentes optimisations proposées par le DataLoader de PyTorch sont accessibles dans le script `dlojz.py` via les arguments :
* `--num-workers <num_workers>` (défaut à `8`)
* `--persistent-workers` (défaut) ou `--no-persistent-workers`
* `--pin-memory` (défaut) ou `--no-pin-memory`
* `--non-blocking` (défaut) ou `--no-non-blocking`
* `--prefetch-factor <prefetch_factor>` (défaut à `2`)
* `--drop-last` ou `--no-drop-last` (défaut)

**TODO** : faire varier ces différents paramètres et observer leurs effets grâce au profiler `turbo_profiler`. Pour comparer les différents essais, ceux-ci seront stockés dans la *DataFrame* `dataloader_trials` initialisée plus tôt.

1. Modifier un ou plusieurs paramètres du DataLoader et lancer l'exécution :

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
#jobid = ['2189183']

In [ ]:
display_slurm_queue(name)

2. Visualiser le retour du turbo profiler :

In [ ]:
# call turbo_profiler
dataloader_trial = turbo_profiler(jobid,dataloader_info=True)

3. Stocker le nouveau résultat dans la DataFrame `dataloader_trials` :

In [ ]:
# store result in "dataloader_trials" DataFrame
dataloader_trials = pd.concat([dataloader_trials,dataloader_trial], ignore_index=True)

4. Visualiser et comparer l'ensemble des résultats :

In [ ]:
# afficher le tableau récapitulatif, trier par ordre croissant du LOADING_TIME
dataloader_trials.sort_values("loading_time").drop_duplicates()

5. Répéter les étapes 1. à 4. jusqu'à avoir trouvé des paramètres d'optimisation satisfaisants.

### Contrôle technique (version optimisée)

**TODO** : relancer l'exécution sur 1 GPU et 100 itérations (`--test-nsteps 100`) sans profiling pour passer un nouveau contrôle technique, à comparer avec celui de référence.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
#jobid = ['2189222']

In [ ]:
display_slurm_queue(name)

In [ ]:
turbo_profiler(jobid)

In [ ]:
controle_technique(jobid)

### OPTIONNEL : Visualisation des traces profiler avec TensorBoard (version sous optimisée)
**TODO** : relancer le job en **réactivant le profiler PyTorch** dans le script [dlojz2_1.py](./dlojz2_1.py) (revoir le TP1_4) afin de visualiser les traces sous TensorBoard, et les comparer avec la version optimisée étudiée dans le TP1_4.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'éviter de relancer un job par erreur.

In [ ]:
#jobid = ['1732254']

In [ ]:
display_slurm_queue(name)

**TODO** : vérifier qu'une trace a bien été générée dans le répertoire `profiler/<name>_<jobid>_bs512_is224/` sous la forme d'un fichier `.json`:

In [ ]:
!tree profiler/

**TODO** : visualiser cette trace grâce à l'application TensorBoard. 

**IMPORTANT** : une fois le TP terminé, penser à quitter l'instance JupyterHub pour **libérer le GPU** ( *> Hub Control Panel > Cancel* ).

![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")

----------------------------------------------
## Garage - Mise à niveau

In [ ]:
image_size = 224
bs_optim = 512

# TP2_2 : Distribution - Parallélisme de données

Voir la [documentation de l'IDRIS](http://www.idris.fr/jean-zay/gpu/jean-zay-gpu-torch-multi.html).

**TODO** : dans le script [dlojz2_2.py](./dlojz2_2.py) :
* Importer les librairies liées à la distribution et au *Data Parallelism*.

* Configurer et initialiser l'environnement parallèle.

* Associer le bon GPU alloué au *process* actif.

* Basculer le modèle en mode *DistributedDataParallelism* pour qu'il soit dupliqué sur les différents GPU.

* Définir les *samplers* distribués `train_sampler` et `val_sampler` et les utiliser dans `train_loader` et `val_loader` respectivement. ***Attention***, le *shuffling* devra être délégué aux samplers.
    
* Au tout début de la boucle d'apprentissage, indiquer au *sampler* l'*epoch* en cours afin d'obtenir un *shuffling* différent à chaque *epoch*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['2189271']

In [ ]:
display_slurm_queue(name)

### Quizz

L'éxécution étant assez longue, un quizz vous attend : [Quizz TP2_2](https://www.deepmama.com/quizz/dlojz_quizz5.html)

In [ ]:
controle_technique(jobid)

### Communications
#### Découverte de comm_profiler
Pour ce TP, nous avons implémenté un profiler maison léger `comm_profiler` basé sur les traces de DEBUG de NCCL pour visualiser la quantité et le type de communications collectives échangées pendant une boucle d'apprentissage distribuée sur plusieurs GPU.

**À noter :** dans le script python [dlojz2_2.py](./dlojz2_2.py) les variables de trace de *DEBUG* *NCCL* sont configurées comme suit :

```python
if __name__ == '__main__':
    
    os.environ["NCCL_DEBUG"] = "INFO"
    os.environ["NCCL_DEBUG_SUBSYS"] = "INIT,COLL"
    # display info
    ...
```

In [ ]:
comm_profiler(jobid, n_display=65)

[from pytorch documentation : ](https://pytorch.org/docs/stable/notes/ddp.html#internal-design)

Each DDP process creates a local `Reducer`, which will take care of the gradients synchronization during the backward pass. To improve communication efficiency, the `Reducer` organizes parameter gradients into **buckets**, and reduces one bucket at a time. **Bucket size** can be configured by setting the bucket_cap_mb argument in DDP constructor. The mapping from parameter gradients to buckets is determined at the construction time, based on the bucket size limit and parameter sizes. 


![buckets](./images/buckets1.png)

### DDP inter-noeud

Nous avons utilisé précédemment **4 GPU** sur le même nœud de calcul. Les bus de communication **intra-nœud** *NVLink* sont très rapide. **Le *scaling* est quasi parfait**.

Si nous utilisons **32 GPU** en *DDP* avec 4 nœuds de calcul et donc des communications sur le réseau d'**interconnexion des nœuds** nous obtenons le résultat suivant.

![DDP 32 GPU](./images/ddp32GPU.png)

**Ce test n'est pas faisable pendant le TP par chacun d'entre vous, pour des raisons évidentes d'accès aux ressources. Veuillez vous reporter au résultat fourni ici.**

![Commentaires](images/cedez.png "Assurez-vous que tout se passe bien avant de continuer!")

### BatchNorm Layer & SyncBatchNorm Layer

**Rappel** :

Pendant l'apprentissage, la couche normalise ses sorties en utilisant la moyenne et l'écart type du batch d'entrée.
Plus exactement, la couche retourne `(batch - mean(batch)) / (var(batch) + epsilon) * weight + bias` , avec :

* `epsilon`, une petite constante pour éviter la division par 0,
* `weight`, un facteur appris (entraîné) avec un calcul de gradient lors de la backpropagation et qui est initialisé à 1,
* `bias`, un facteur appris (entraîné) avec un calcul de gradient lors de la backpropagation et qui est initialisé à 0.

Pendant l'inférence ou la validation, la couche normalise ses sorties en utilisant en plus des `weight` et `bias` entraînés, les facteurs `running_mean` et `running_var` : `(batch - running_mean) / (running_var + epsilon) * weight + bias`.

`running_mean` et `running_var` sont des facteurs non entraînés, mais qui sont mis à jour à chaque itération de batch lors de l'apprentissage, selon la méthode suivante :

* `running_mean = running_mean * momentum + mean(batch) * (1 - momentum)`
* `running_var = running_var * momentum + var(batch) * (1 - momentum)`


In [ ]:
import torchvision.models as models
model = models.resnet152()

In [ ]:
BatchNorm_view(jobid, model)

### SyncBatchNorm layer
Voir la [documentation PyTorch](http://www.idris.fr/ia/syncbn.html#syncbn_en_pytorch).

**TODO** : dans le script [dlojz2_2.py](./dlojz2_2.py) :
* Juste avant la bascule du modèle en mode *DistributedDataParallelism*, transformer les couches *BatchNorm* du modèle en couches *SyncBatchNorm*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid_sync = ['2189317']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid_sync)

In [ ]:
BatchNorm_view(jobid + jobid_sync, model, labels=['BN Layer', 'SyncBN Layers'])

#### Communications

In [ ]:
comm_profiler(jobid_sync, n_display=100)

![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")